PHASE 1: ASK



Business Objective: Conduct a comprehensive analysis of the sales performance of an online store, broken down by products, periods of time, and customers. From this analysis, the aim is to devise effective strategies to boost sales and optimize future inventory management.

In [ ]:
# Guiding questions:

# Which are the most sold products?

# Which products generate the most revenue?

# In which months, days, hours of day, are the most revenue generated? 

# Who are the most valuable customers? 

PHASE 2: PREPARE

In [ ]:
# Importing the libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Creating the data frame
sales = pd.read_csv("C:/Users/adria/OneDrive/Documentos/Sales_proyect/Online_Retail.csv", encoding='latin1')

In [ ]:
# Checking data types
print(sales.dtypes)

In [47]:
print(sales.describe())
# Outliers were examined in Excel by appplying filters. 

            Quantity                    InvoiceDate      UnitPrice  \
count  541909.000000                         541909  541909.000000   
mean        9.552250  2011-07-04 13:34:57.156386048       4.611114   
min    -80995.000000            2010-12-01 08:26:00  -11062.060000   
25%         1.000000            2011-03-28 11:34:00       1.250000   
50%         3.000000            2011-07-19 17:17:00       2.080000   
75%        10.000000            2011-10-19 11:27:00       4.130000   
max     80995.000000            2011-12-09 12:50:00   38970.000000   
std       218.081158                            NaN      96.759853   

          CustomerID  
count  406829.000000  
mean    15287.690570  
min     12346.000000  
25%     13953.000000  
50%     15152.000000  
75%     16791.000000  
max     18287.000000  
std      1713.600303  


PHASE 3: CLEANING

In [ ]:
# In order to work with complete months, it is necessary to delimit the sales from 1st December 2010 to 30th November 2011

# First, conversion of column InvoiceDate from object type to datetime type
sales['InvoiceDate'] = pd.to_datetime(sales['InvoiceDate'], format='%d/%m/%Y %H:%M')

# Now, the filter can be applied
sales_clean = sales[sales['InvoiceDate'].between('2010-12-01', '2011-11-30')]

In [ ]:
# I assume most of the NaN values in column CustomerID are uncompleted purchases, so them won´t be considered.
# The rest of NaN values seem to be missing products, them also won´t be considered for this project.

# Delete of NaN values in CustomerID column
sales_clean = sales_clean.dropna(subset=['CustomerID'])

In [ ]:
# Count of NaN values before cleaning
nan_count = sales.isna().sum().sum()
print(f'The total amount of NaN values in CustomerID before cleaning is: {nan_count}')

# Count of NaN values after cleaning
nan_count_2 = sales_clean.isna().sum().sum()
print(f'The total amount of NaN values in CustomerID after cleaning is:{nan_count_2}')

In [ ]:
# Creating new columns
sales_clean['Revenue'] = sales_clean['Quantity'] * sales['UnitPrice'] #Revenue column

sales_clean['InvoiceMonth'] = sales_clean['InvoiceDate'].dt.month     #InvoiceMonth column

sales_clean['InvoiceDay'] = sales_clean['InvoiceDate'].dt.day_name()   #InvoiceDay column

sales_clean['HourofDay'] = sales_clean['InvoiceDate'].dt.hour          #HourofDay column

In [ ]:
# Rearrenging the order of columns
new_order = ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'UnitPrice', 'Revenue', 'InvoiceDate', 'InvoiceMonth', 'InvoiceDay', 'HourofDay', 'Country', 'CustomerID',]
sales_clean = sales_clean[new_order]

PHASE  4 & 5: ANALYSIS AND SHARE

In [ ]:
 # Analysis by time

# Monthly revenue 
monthly_revenue = sales_clean.groupby('InvoiceMonth')['Revenue'].sum().sort_values(ascending=False)
print(monthly_revenue)

# Plot
monthly_revenue = monthly_revenue.sort_index()
monthly_revenue.plot(kind='bar')
plt.title('Monthly Revenue')
plt.ylabel('Revenue')
plt.show()

 # Daily revenue 
daily_revenue = sales_clean.groupby('InvoiceDay')['Revenue'].sum().sort_values(ascending=False)
print(daily_revenue)

# Plot
order_of_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Sunday']
daily_revenue = daily_revenue.loc[order_of_days]
daily_revenue.plot(kind='bar')
plt.title('Daily Revenue')
plt.ylabel('Revenue')
plt.show()

# Revenue by hour of day
byhour_revenue = sales_clean.groupby('HourofDay')['Revenue'].sum().sort_values(ascending=False)
print(byhour_revenue)

# Plot
byhour_revenue = byhour_revenue.sort_index()
byhour_revenue.plot(kind='bar')
plt.title('Revenue by Hour of Day')
plt.ylabel('Revenue')
plt.show()

In [ ]:
# Analysis of products

# Most profitable products
most_profitable = sales_clean.groupby(['Description','StockCode'])['Revenue'].sum().sort_values(ascending=False).head(10)
print(most_profitable)

# Plot
most_profitable.plot(kind='bar')
plt.title('Most Profitable Products')
plt.ylabel('Revenue')
plt.show()

# Most sold products
most_sold = sales_clean.groupby(['Description','StockCode'])['Quantity'].sum().sort_values(ascending=False).head(10)
print(most_sold)

# Plot
most_sold.plot(kind='bar')
plt.title('Most Sold Products')
plt.ylabel('Quantity')
plt.show()

# Products in both rankings
products_both_rankings = most_sold.index.intersection(most_profitable.index)
print(products_both_rankings)

In [ ]:
# Analysis RFM (customer behaivor)

# Recency: days since last purchase
snapshot_date = sales_clean['InvoiceDate'].max() + pd.Timedelta(days=1)
rfm = sales_clean.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'count',
    'Revenue': 'sum'
})

# Rename columns
rfm.columns = ['Recency', 'Frequency', 'Monetary']

# Clasification of RFM
rfm['R_Score'] = pd.qcut(rfm['Recency'], 4, labels=range(4, 0, -1))
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 4, labels=range(1, 5))
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 4, labels=range(1, 5))

# RFM Segment
rfm['RFM_Segment'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)
rfm['RFM_Score'] = rfm[['R_Score', 'F_Score', 'M_Score']].sum(axis=1)

# Agrupation by segment
rfm_segments = rfm.groupby('RFM_Segment').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': ['mean', 'count']
}).round(1)
print(rfm_segments)

# Plot
plt.figure(figsize=(12, 8))
sns.scatterplot(x='Recency', y='Monetary', size='Frequency', data=rfm)
plt.title('Análisis RFM - Scatterplot')
plt.xlabel('Recency')
plt.ylabel('Monetary')
plt.show()


In [ ]:
# Key findings

# The highest revenue-generating months are September, October, and November, while the most profitable days of the week are Tuesday and Thursday, and the peak hours for revenue generation are from 10:00 to 13:59.

# Products wich are both in most sold products and most profitable products are: JUMBO BAG RED RETROSPOT, ASSORTED COLOUR BIRD ORNAMENT,
# WHITE HANGING HEART T-LIGHT HOLDER and RABBIT NIGHT LIGHT.

# High-Value Customers are (RFM Segment = 444, 434), and At-Risk Customers are (RFM Segment = 111, 112, 113).

PHASE 6: ACT

In [ ]:
# The top three recomendations to boost sales and optimize future inventory managment are: ......